In [7]:
cd

C:\Users\HP


In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\HP'

In [3]:
path='C:\\Users\\HP\\Desktop'

In [4]:
os.chdir(path)

In [5]:
os.getcwd()

'C:\\Users\\HP\\Desktop'

In [6]:
import csv

In [7]:
with open('TreasuryYieldCrv20200212.csv','r') as csvFile1:
    reader=csv.reader(csvFile1)
    r=list(reader)

In [8]:
print(r)

[['T', 'Yield'], ['0.083333333', '1.57'], ['0.166666667', '1.59'], ['0.25', '1.58'], ['0.5', '1.56'], ['1', '1.49'], ['2', '1.44'], ['3', '1.42'], ['5', '1.45'], ['7', '1.55'], ['10', '1.62'], ['20', '1.93'], ['30', '2.09']]


In [9]:
column1=[]
column2=[]

In [10]:
for row in range(len(r)):
    column1.append(r[row][0])
    column2.append(r[row][1])

In [20]:
class Curve_base:
    def __init__(self,x,y):
        #make sure you check x and y list have the same length using exception
        if  len(x)!=len(y):
            
            raise Exception('The length of two data should be the same!!')
            
        self.x=x
        self.y=y
        number=len(x)
        
            
    def _str_(self):
        return{'Maturity':self.x[1:],'Yield':self.y[1:]}
    def _repr_(self):
        return{'Maturity':str(self.x[1:]),'Yield':str(self.y[1:])}
    def _call_(self,xValue):
        #print(__name__)
        for index,v in enumerate(self.x[1:]):
            if float(v)>xValue:
                mark=index
                break;
                #print(mark)
                #find the biggest number's index that smaller than the given xValue
        temp_y=float(self.y[mark])+(float(self.y[mark+1])-float(self.y[mark]))*(xValue-float(self.x[mark]))/(float(self.x[mark+1])-float(self.x[mark]))
        #linearinterprepolation
        return  print('interpolated yield is '+str(temp_y))    
            

In [21]:
C1=Curve_base(column1,column2)

In [23]:
class FwdYldCurve(Curve_base):
    def __init__(self,x,y):
        #get the constructor from the base
        super().__init__(x,y)

In [63]:
class ParYldCurve(Curve_base):
    def __init__(self,x,y):
        super().__init__(x,y)
    def fwdRate(self,t,T,TT):
        #current time:t------------Time T's spot rate:r---------------Time TT's spot rate :rTT-------
        #rF:at current time t,the forward rate of time from T to TT
        
            if(t>T or T>TT):
                raise Exception('woops, your time set is wrong')
            try:
                for index,v in  enumerate(self.x[1:]):
                    if float(v)==t:
                        index1=index+1
                    if float(v)==T:
                        index2=index+1
                    if float(v)==TT:
                        index3=index+1
            #formula is exp(r*(T-t))*exp(rF*(TT-T))=exp(rTT(TT-t))
                rF=(float(self.y[index3])*(float(self.x[index3])-float(self.x[index1]))-float(self.y[index2])*(float(self.x[index2])-float(self.x[index1])))/(float(self.x[index3])-float(self.x[index2]))
                return('Forward Rate is '+str(rF))                                                                                                                                          
            except(UnboundLocalError) as e :
                #make sure every input can be found in csv 
                    print("you have inputed a number(T) which doesn't exist in the csv:"+str(e))
                    

In [107]:
#test if this python program is being run by itself
__name__

'__main__'

In [94]:
par=ParYldCurve(column1,column2)


In [95]:
column_for_fwd1=[]
column_for_fwd2=[]

In [97]:
for index1,v1 in enumerate(par.x[1:]):
    for index2,v2 in enumerate(par.x[index1+2:]):
        for index3,v3 in enumerate(par.x[index2+index1+3:]):
            column_for_fwd1.append('at current time :'+str(v1)+' ,the forward rate from time '+str(v2)+' to time '+str(v3)+' is ')
            column_for_fwd2.append(str(par.fwdRate(float(v1),float(v2),float(v3))))
            print('at current time :'+str(v1)+' ,the forward rate from time '+str(v2)+' to time '+str(v3)+' is '+str(par.fwdRate(float(v1),float(v2),float(v3))))


at current time :0.083333333 ,the forward rate from time 0.166666667 to time 0.25 is Forward Rate is 1.5699999998799998
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 0.5 is Forward Rate is 1.5524999999324998
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 1 is Forward Rate is 1.479999999916
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 2 is Forward Rate is 1.433181818126033
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 3 is Forward Rate is 1.4149999999594116
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 5 is Forward Rate is 1.447586206877075
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 7 is Forward Rate is 1.549512195118025
at current time :0.083333333 ,the forward rate from time 0.166666667 to time 10 is Forward Rate is 1.620254237290178
at current time :0.083333333 ,the forward rate from time 0.1666666

In [105]:
#Get the FwdYldCurve object from that ParYldCurve
fwd=FwdYldCurve(column_for_fwd1,column_for_fwd2)

In [99]:
#zip two list into one to put into csv file as columns
zipped=zip( column_for_fwd1,column_for_fwd2)

In [100]:
listzip=list(zipped)

In [111]:
#add newline='' to avoid the blank row
with open('20200212TreasuryFwdCrv.csv','w',newline='') as f:
    writer=csv.writer(f)
    for row in listzip:
        writer.writerow(row)

In [108]:
with open('result.out','w') as f:
    f.write('The “interpolated” par yield for the list of maturities: [0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.25, 1.5]')